In [33]:
import numpy as np
import pandas as pd
import random

In [34]:
# new data to combine with initial data (Note: formatted differently from df_2018 data)
df_ks = pd.read_csv("../Data/kickstarter.csv")

In [35]:
# Cleaning
# 1. Delete the first column (i.e., the row id 'Unnamed')
del df_ks['Unnamed: 0']

In [36]:
# 2. Replace state string with numbers in df_sk
cleanup_nums = {"state": {"failed": 0, "successful": 1, "canceled" : -1, "undefined" : -1, "live" : -1, "suspended" : -1}}
df_ks.replace(cleanup_nums, inplace=True)

In [37]:
# 3. Remove Duplicate values
# 3a. Sort by the number of backers (assuming you'll have more backers)
df_ks_sorted = df_ks.sort_values(by='backers_count')
# 3b. Remove duplicates
df_ks_sorted.drop_duplicates(subset='id', keep='last', inplace=True)

In [38]:
# 4. Create new dataframe with the columns (that are not in 2018) = 
# id, backers_count, blurb, blurb_length, spotlight, staff_pick, location_state, location_displayable_name
df_ks_dropped = df_ks_sorted[['id','backers_count', 'blurb', 'blurb_length', 'spotlight', 'staff_pick', 'location_state', 'location_displayable_name']].sort_values(by='id')

In [39]:
df_ks_dropped.head()

,id,backers_count,blurb,blurb_length,spotlight,staff_pick,location_state,location_displayable_name
343901,8624,737,A compact outdoor cooking set that you can use...,12,True,True,WV,"Belgium, WV"
212211,18520,4,Raising money to help my grandmother recover f...,24,False,False,OH,"Cleveland, OH"
11159,21109,11,My work is performance based but I branch out ...,24,True,False,Scotland,"Dundee, UK"
62717,24380,18,A sanctuary for humans and felines alike! Come...,24,False,False,IL,"Bloomington, IL"
34144,33867,42,Taste Makers is a socially conscious brand tha...,23,False,True,BC,"Vancouver, Canada"


In [40]:
# Read in other Data files
df_2018 = pd.read_csv("../Data/Cleaned/2018.csv", parse_dates= ['launched', 'deadline'])
# Rename ID to id
df_2018 = df_2018.rename(columns={"ID": "id"})

In [41]:
def time_since_last_project(series):
    # Return the time in hours
    return series.diff().dt.total_seconds() / 3600.

df = df_2018[['category', 'launched']].sort_values('launched')
timedeltas = df.groupby('category').transform(time_since_last_project)
timedeltas.head(20)

,launched
148129,NaN
281872,NaN
121357,NaN
284836,NaN
107424,137.130833
187060,NaN
302546,145.941111
205965,NaN
299605,NaN
240887,NaN


In [42]:
# Final time since last project (in hours)
timedeltas = timedeltas.fillna(timedeltas.median()).reindex(df_ks.index)
timedeltas.head(10)

,launched
0,189.000278
1,5.592778
2,1.313611
3,0.635000
4,2.629722
5,0.367500
6,12.286111
7,1.372222
8,8.524444
9,0.015833


In [43]:
df_2018['hours_since_last_project'] = timedeltas['launched']
df_2018.head()

,id,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,hours_since_last_project
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,0,0,GB,0.0,0.0,1533.95,189.000278
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,0,15,US,100.0,2421.0,30000.00,5.592778
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,0,3,US,220.0,220.0,45000.00,1.313611
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,0,1,US,1.0,1.0,5000.00,0.635000
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.0,1,224,US,52375.0,52375.0,50000.00,2.629722


In [44]:
# Join the two tables on id
df_joined_ids = df_ks_dropped.set_index('id').join(df_2018.set_index('id'))

In [45]:
# Remove any rows that have a NaN in the name column
df_joined_ids = df_joined_ids.dropna();
df_joined_ids.head()

,backers_count,blurb,blurb_length,spotlight,staff_pick,location_state,location_displayable_name,name,category,main_category,...,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,hours_since_last_project
id,,,,,,,,,,,,,,,,,,,,,
18520,4,Raising money to help my grandmother recover f...,24,False,False,OH,"Cleveland, OH",Grandma's are Life,World Music,Music,...,15000.0,2016-10-19 15:32:40,62.0,0.0,4.0,US,0.00,62.00,15000.00,66.078889
21109,11,My work is performance based but I branch out ...,24,True,False,Scotland,"Dundee, UK",Meta,Performance Art,Art,...,150.0,2015-04-08 00:37:44,173.0,1.0,11.0,GB,258.04,266.65,231.20,29.557500
24380,18,A sanctuary for humans and felines alike! Come...,24,False,False,IL,"Bloomington, IL",Puss N' Books: A relaxing cat cafe and bookstore.,Spaces,Food,...,20000.0,2015-10-27 16:25:33,776.0,0.0,18.0,US,776.00,776.00,20000.00,113.248611
33867,42,Taste Makers is a socially conscious brand tha...,23,False,True,BC,"Vancouver, Canada",TASTE MAKERS BY TRISH P,Ready-to-wear,Fashion,...,18000.0,2015-06-15 20:28:11,2798.0,0.0,42.0,CA,2271.72,2144.39,13795.22,2.419444
39036,68,"The BEST beef sticks, beef jerky and signature...",13,True,False,WI,"Kronenwetter, WI",The Meat Candy Experience,Small Batch,Food,...,2500.0,2016-05-17 00:34:18,3239.0,1.0,68.0,US,3239.00,3239.00,2500.00,3.911389


In [46]:
# Convert Timestamps
df_joined_ids = df_joined_ids.assign(
                launch_hour=df_joined_ids.launched.dt.hour,
                launch_day=df_joined_ids.launched.dt.day,
                launch_month=df_joined_ids.launched.dt.month,
                launch_year=df_joined_ids.launched.dt.year)

df_joined_ids = df_joined_ids.assign(
                deadline_day=df_joined_ids.deadline.dt.day,
                deadline_month=df_joined_ids.deadline.dt.month,
                deadline_year=df_joined_ids.deadline.dt.year)

In [47]:
# Add column: project duration in days
#from datetime import datetime
df_joined_ids['launched'] = pd.to_datetime(df_joined_ids['launched']).dt.date
df_joined_ids['deadline'] = pd.to_datetime(df_joined_ids['deadline']).dt.date

df_joined_ids['duration_days'] = (df_joined_ids['deadline'] - df_joined_ids['launched']).dt.days

In [48]:
df_joined_ids.head(3)

,backers_count,blurb,blurb_length,spotlight,staff_pick,location_state,location_displayable_name,name,category,main_category,...,usd_goal_real,hours_since_last_project,launch_hour,launch_day,launch_month,launch_year,deadline_day,deadline_month,deadline_year,duration_days
id,,,,,,,,,,,,,,,,,,,,,
18520,4,Raising money to help my grandmother recover f...,24,False,False,OH,"Cleveland, OH",Grandma's are Life,World Music,Music,...,15000.0,66.078889,15,19,10,2016,18,11,2016,30
21109,11,My work is performance based but I branch out ...,24,True,False,Scotland,"Dundee, UK",Meta,Performance Art,Art,...,231.2,29.557500,0,8,4,2015,6,5,2015,28
24380,18,A sanctuary for humans and felines alike! Come...,24,False,False,IL,"Bloomington, IL",Puss N' Books: A relaxing cat cafe and bookstore.,Spaces,Food,...,20000.0,113.248611,16,27,10,2015,26,11,2015,30


In [49]:
# Drop Redundant Columns
df_joined_ids = df_joined_ids.drop(columns=['location_displayable_name'])
df_joined_ids = df_joined_ids.drop(columns=['backers'])
df_joined_ids = df_joined_ids.drop(columns=['usd pledged'])
df_joined_ids = df_joined_ids.drop(columns=['currency'])
df_joined_ids = df_joined_ids.drop(columns=['pledged'])
df_joined_ids = df_joined_ids.drop(columns=['goal'])
#df_joined_ids = df_joined_ids.drop(columns=['launched'])
#df_joined_ids = df_joined_ids.drop(columns=['deadline'])

In [50]:
df_joined_ids.head()

,backers_count,blurb,blurb_length,spotlight,staff_pick,location_state,name,category,main_category,deadline,...,usd_goal_real,hours_since_last_project,launch_hour,launch_day,launch_month,launch_year,deadline_day,deadline_month,deadline_year,duration_days
id,,,,,,,,,,,,,,,,,,,,,
18520,4,Raising money to help my grandmother recover f...,24,False,False,OH,Grandma's are Life,World Music,Music,2016-11-18,...,15000.00,66.078889,15,19,10,2016,18,11,2016,30
21109,11,My work is performance based but I branch out ...,24,True,False,Scotland,Meta,Performance Art,Art,2015-05-06,...,231.20,29.557500,0,8,4,2015,6,5,2015,28
24380,18,A sanctuary for humans and felines alike! Come...,24,False,False,IL,Puss N' Books: A relaxing cat cafe and bookstore.,Spaces,Food,2015-11-26,...,20000.00,113.248611,16,27,10,2015,26,11,2015,30
33867,42,Taste Makers is a socially conscious brand tha...,23,False,True,BC,TASTE MAKERS BY TRISH P,Ready-to-wear,Fashion,2015-07-30,...,13795.22,2.419444,20,15,6,2015,30,7,2015,45
39036,68,"The BEST beef sticks, beef jerky and signature...",13,True,False,WI,The Meat Candy Experience,Small Batch,Food,2016-07-01,...,2500.00,3.911389,0,17,5,2016,1,7,2016,45


In [51]:
df_joined_ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242630 entries, 18520 to 2147476221
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   backers_count             242630 non-null  int64  
 1   blurb                     242630 non-null  object 
 2   blurb_length              242630 non-null  int64  
 3   spotlight                 242630 non-null  bool   
 4   staff_pick                242630 non-null  bool   
 5   location_state            242630 non-null  object 
 6   name                      242630 non-null  object 
 7   category                  242630 non-null  object 
 8   main_category             242630 non-null  object 
 9   deadline                  242630 non-null  object 
 10  launched                  242630 non-null  object 
 11  state                     242630 non-null  float64
 12  country                   242630 non-null  object 
 13  usd_pledged_real          242630 non

In [52]:
df_joined_ids.to_csv("../Data/Cleaned/joined_data.csv", index = False)